In [15]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import cartopy.io.shapereader as shpreader
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.patches as mpatches
import geopandas as gpd

import psycopg2
from sqlalchemy import create_engine

https://www.census.gov/topics/public-sector/voting/data/tables.2010.List_1863097513.html#list-tab-List_1863097513

In [16]:
# read in the Excel file and specify the sheet name
df_votes_2016 = pd.read_excel('Election_Results/vote2016.xlsx', sheet_name='Table 4a', header=[4])

# display the resulting dataframe
df_votes_2016

Table 4a


Unnamed: 0  Unnamed: 1  Unnamed: 2   
0                                       UNITED STATES    245502.0    224059.0  \
1                                             ALABAMA      3717.0      3651.0   
2                                              ALASKA       518.0       502.0   
3                                             ARIZONA      5196.0      4585.0   
4                                            ARKANSAS      2216.0      2116.0   
5                                          CALIFORNIA     29894.0     24890.0   
6                                            COLORADO      4242.0      3895.0   
7                                         CONNECTICUT      2759.0      2483.0   
8                                            DELAWARE       729.0       669.0   
9                                DISTRICT OF COLUMBIA       553.0       512.0   
10                                            FLORIDA     16202.0     14428.0   
11                                            GEORGIA      7626.0      7048.0   
12                                             HAWAII      1064.0       974.0   
13                                              IDAHO      1224.0      1150.0   
14                                           ILLINOIS      9723.0      8970.0   
15                                            INDIANA      4988.0      4795.0   
16                                               IOWA      2394.0      2292.0   
17                                             KANSAS      2142.0      2029.0   
18                                           KENTUCKY      3348.0      3246.0   
19                                          LOUISIANA      3463.0      3353.0   
20                                              MAINE      1058.0      1038.0   
21                                           MARYLAND      4623.0      4158.0   
22                                      MASSACHUSETTS      5374.0      4967.0   
23                                           MICHIGAN      7624.0      7332.0   
24                                          MINNESOTA      4190.0      3985.0   
25                                        MISSISSIPPI      2203.0      2170.0   
26                                           MISSOURI      4626.0      4486.0   
27                                            MONTANA       798.0       790.0   
28                                           NEBRASKA      1407.0      1336.0   
29                                             NEVADA      2234.0      1975.0   
30                                      NEW HAMPSHIRE      1044.0      1012.0   
31                                         NEW JERSEY      6862.0      5958.0   
32                                         NEW MEXICO      1547.0      1396.0   
33                                           NEW YORK     15506.0     13751.0   
34                                     NORTH CAROLINA      7631.0      6960.0   
35                                       NORTH DAKOTA       583.0       564.0   
36                                               OHIO      8811.0      8499.0   
37                                           OKLAHOMA      2923.0      2746.0   
38                                             OREGON      3185.0      2929.0   
39                                       PENNSYLVANIA      9980.0      9596.0   
40                                       RHODE ISLAND       836.0       766.0   
41                                     SOUTH CAROLINA      3733.0      3598.0   
42                                       SOUTH DAKOTA       631.0       612.0   
43                                          TENNESSEE      5057.0      4872.0   
44                                              TEXAS     20172.0     17378.0   
45                                               UTAH      2096.0      1969.0   
46                                            VERMONT       500.0       488.0   
47                                           VIRGINIA      6343.0      5829.0   
48                                         WASHINGTON      5592.0      510

In [17]:
# select the columns wanted and rename them
df_selection = df_votes_2016[['Unnamed: 0', 'Unnamed: 1', 'Total registered', 'Total voted']]
df_votes = df_selection.rename(columns={
    'Unnamed: 0': 'State',
    'Unnamed: 1': 'Population',
    'Total registered': 'Registered',
    'Total voted': 'Voted'
})
# add year and office columns
df_votes['Year'] = 2016

# Arrange columns: Move the Year and Office columns to the front
new_cols = ['Year', 'State', 'Population', 'Registered', 'Voted']

# df_votes = df_votes.iloc[1:]  # remove the first row
df_votes = df_votes.iloc[:-3] # removes last 7 rows of text

# Save the first row as a separate DataFrame
first_row = df_votes.iloc[[0]].copy()

# Remove the first row from the original DataFrame
df_votes = df_votes.iloc[1:].copy()

# Append the first row to the end of the DataFrame
df_votes = pd.concat([df_votes, first_row])

df_votes = df_votes.reset_index(drop=True)  # reset the index
df_votes = df_votes.reindex(columns=new_cols) # set year column in front

df_votes
# values are in the thousands

Year                 State  Population  Registered     Voted
0   2016               ALABAMA      3717.0      2526.0    2095.0
1   2016                ALASKA       518.0       358.0     308.0
2   2016               ARIZONA      5196.0      3145.0    2769.0
3   2016              ARKANSAS      2216.0      1456.0    1241.0
4   2016            CALIFORNIA     29894.0     16096.0   14416.0
5   2016              COLORADO      4242.0      2893.0    2707.0
6   2016           CONNECTICUT      2759.0      1763.0    1586.0
7   2016              DELAWARE       729.0       487.0     417.0
8   2016  DISTRICT OF COLUMBIA       553.0       420.0     380.0
9   2016               FLORIDA     16202.0      9604.0    8578.0
10  2016               GEORGIA      7626.0      4892.0    4246.0
11  2016                HAWAII      1064.0       530.0     460.0
12  2016                 IDAHO      1224.0       790.0     714.0
13  2016              ILLINOIS      9723.0      6665.0    5719.0
14  2016               INDIANA      4988.0      3298.0    2795.0
15  2016                  IOWA      2394.0      1657.0    1454.0
16  2016                KANSAS      2142.0      1438.0    1243.0
17  2016              KENTUCKY      3348.0      2253.0    1850.0
18  2016             LOUISIANA      3463.0      2446.0    2067.0
19  2016                 MAINE      1058.0       830.0     754.0
20  2016              MARYLAND      4623.0      3114.0    2737.0
21  2016         MASSACHUSETTS      5374.0      3660.0    3315.0
22  2016              MICHIGAN      7624.0      5434.0    4713.0
23  2016             MINNESOTA      4190.0      3055.0    2738.0
24  2016           MISSISSIPPI      2203.0      1725.0    1470.0
25  2016              MISSOURI      4626.0      3333.0    2906.0
26  2016               MONTANA       798.0       581.0     521.0
27  2016              NEBRASKA      1407.0      1008.0     893.0
28  2016                NEVADA      2234.0      1371.0    1195.0
29  2016         NEW HAMPSHIRE      1044.0       763.0     698.0
30  2016            NEW JERSEY      6862.0      4165.0    3665.0
31  2016            NEW MEXICO      1547.0       916.0     765.0
32  2016              NEW YORK     15506.0      9142.0    7869.0
33  2016        NORTH CAROLINA      7631.0      5194.0    4700.0
34  2016          NORTH DAKOTA       583.0       424.0     362.0
35  2016                  OHIO      8811.0      6128.0    5408.0
36  2016              OKLAHOMA      2923.0      1861.0    1555.0
37  2016                OREGON      3185.0      2147.0    1942.0
38  2016          PENNSYLVANIA      9980.0      6909.0    6008.0
39  2016          RHODE ISLAND       836.0       538.0     464.0
40  2016        SOUTH CAROLINA      3733.0      2575.0    2233.0
41  2016          SOUTH DAKOTA       631.0       437.0     362.0
42  2016             TENNESSEE      5057.0      3251.0    2630.0
43  2016                 TEXAS     20172.0     11724.0    9626.0
44  2016                  UTAH      2096.0      1398.0    1234.0
45  2016               VERMONT       500.0       351.0     305.0
46  2016              VIRGINIA      6343.0      4399.0    3973.0
47  2016            WASHINGTON      5592.0      3906.0    3382.0
48  2016         WEST VIRGINIA      1434.0       913.0     723.0
49  2016             WISCONSIN      4465.0      3323.0    3068.0
50  2016               WYOMING       436.0       304.0     277.0
51  2016         UNITED STATES    245502.0    157596.0  137537.0

In [18]:
state_abbr = {
    'ALABAMA': 'AL',
    'ALASKA': 'AK',
    'ARIZONA': 'AZ',
    'ARKANSAS': 'AR',
    'CALIFORNIA': 'CA',
    'COLORADO': 'CO',
    'CONNECTICUT': 'CT',
    'DELAWARE': 'DE',
    'DISTRICT OF COLUMBIA': 'DC',
    'FLORIDA': 'FL',
    'GEORGIA': 'GA',
    'HAWAII': 'HI',
    'IDAHO': 'ID',
    'ILLINOIS': 'IL',
    'INDIANA': 'IN',
    'IOWA': 'IA',
    'KANSAS': 'KS',
    'KENTUCKY': 'KY',
    'LOUISIANA': 'LA',
    'MAINE': 'ME',
    'MARYLAND': 'MD',
    'MASSACHUSETTS': 'MA',
    'MICHIGAN': 'MI',
    'MINNESOTA': 'MN',
    'MISSISSIPPI': 'MS',
    'MISSOURI': 'MO',
    'MONTANA': 'MT',
    'NEBRASKA': 'NE',
    'NEVADA': 'NV',
    'NEW HAMPSHIRE': 'NH',
    'NEW JERSEY': 'NJ',
    'NEW MEXICO': 'NM',
    'NEW YORK': 'NY',
    'NORTH CAROLINA': 'NC',
    'NORTH DAKOTA': 'ND',
    'OHIO': 'OH',
    'OKLAHOMA': 'OK',
    'OREGON': 'OR',
    'PENNSYLVANIA': 'PA',
    'RHODE ISLAND': 'RI',
    'SOUTH CAROLINA': 'SC',
    'SOUTH DAKOTA': 'SD',
    'TENNESSEE': 'TN',
    'TEXAS': 'TX',
    'UTAH': 'UT',
    'VERMONT': 'VT',
    'VIRGINIA': 'VA',
    'WASHINGTON': 'WA',
    'WEST VIRGINIA': 'WV',
    'WISCONSIN': 'WI',
    'WYOMING': 'WY',
    'UNITED STATES': 'US'
}



In [19]:
df_votes['State'] = df_votes['State'].map(state_abbr)
df_votes

Year State  Population  Registered     Voted
0   2016    AL      3717.0      2526.0    2095.0
1   2016    AK       518.0       358.0     308.0
2   2016    AZ      5196.0      3145.0    2769.0
3   2016    AR      2216.0      1456.0    1241.0
4   2016    CA     29894.0     16096.0   14416.0
5   2016    CO      4242.0      2893.0    2707.0
6   2016    CT      2759.0      1763.0    1586.0
7   2016    DE       729.0       487.0     417.0
8   2016    DC       553.0       420.0     380.0
9   2016    FL     16202.0      9604.0    8578.0
10  2016    GA      7626.0      4892.0    4246.0
11  2016    HI      1064.0       530.0     460.0
12  2016    ID      1224.0       790.0     714.0
13  2016    IL      9723.0      6665.0    5719.0
14  2016    IN      4988.0      3298.0    2795.0
15  2016    IA      2394.0      1657.0    1454.0
16  2016    KS      2142.0      1438.0    1243.0
17  2016    KY      3348.0      2253.0    1850.0
18  2016    LA      3463.0      2446.0    2067.0
19  2016    ME      1058.0       830.0     754.0
20  2016    MD      4623.0      3114.0    2737.0
21  2016    MA      5374.0      3660.0    3315.0
22  2016    MI      7624.0      5434.0    4713.0
23  2016    MN      4190.0      3055.0    2738.0
24  2016    MS      2203.0      1725.0    1470.0
25  2016    MO      4626.0      3333.0    2906.0
26  2016    MT       798.0       581.0     521.0
27  2016    NE      1407.0      1008.0     893.0
28  2016    NV      2234.0      1371.0    1195.0
29  2016    NH      1044.0       763.0     698.0
30  2016    NJ      6862.0      4165.0    3665.0
31  2016    NM      1547.0       916.0     765.0
32  2016    NY     15506.0      9142.0    7869.0
33  2016    NC      7631.0      5194.0    4700.0
34  2016    ND       583.0       424.0     362.0
35  2016    OH      8811.0      6128.0    5408.0
36  2016    OK      2923.0      1861.0    1555.0
37  2016    OR      3185.0      2147.0    1942.0
38  2016    PA      9980.0      6909.0    6008.0
39  2016    RI       836.0       538.0     464.0
40  2016    SC      3733.0      2575.0    2233.0
41  2016    SD       631.0       437.0     362.0
42  2016    TN      5057.0      3251.0    2630.0
43  2016    TX     20172.0     11724.0    9626.0
44  2016    UT      2096.0      1398.0    1234.0
45  2016    VT       500.0       351.0     305.0
46  2016    VA      6343.0      4399.0    3973.0
47  2016    WA      5592.0      3906.0    3382.0
48  2016    WV      1434.0       913.0     723.0
49  2016    WI      4465.0      3323.0    3068.0
50  2016    WY       436.0       304.0     277.0
51  2016    US    245502.0    157596.0  137537.0

In [20]:
# Calculate the normalized voter count
df_votes['Norm_Voter_Reg'] = df_votes['Voted'] / df_votes['Registered']
df_votes['Norm_Voter_Pop'] = df_votes['Voted'] / df_votes['Population']
df_votes

Year State  Population  Registered     Voted  Norm_Voter_Reg   
0   2016    AL      3717.0      2526.0    2095.0        0.829375  \
1   2016    AK       518.0       358.0     308.0        0.860335   
2   2016    AZ      5196.0      3145.0    2769.0        0.880445   
3   2016    AR      2216.0      1456.0    1241.0        0.852335   
4   2016    CA     29894.0     16096.0   14416.0        0.895626   
5   2016    CO      4242.0      2893.0    2707.0        0.935707   
6   2016    CT      2759.0      1763.0    1586.0        0.899603   
7   2016    DE       729.0       487.0     417.0        0.856263   
8   2016    DC       553.0       420.0     380.0        0.904762   
9   2016    FL     16202.0      9604.0    8578.0        0.893170   
10  2016    GA      7626.0      4892.0    4246.0        0.867948   
11  2016    HI      1064.0       530.0     460.0        0.867925   
12  2016    ID      1224.0       790.0     714.0        0.903797   
13  2016    IL      9723.0      6665.0    5719.0        0.858065   
14  2016    IN      4988.0      3298.0    2795.0        0.847483   
15  2016    IA      2394.0      1657.0    1454.0        0.877489   
16  2016    KS      2142.0      1438.0    1243.0        0.864395   
17  2016    KY      3348.0      2253.0    1850.0        0.821127   
18  2016    LA      3463.0      2446.0    2067.0        0.845053   
19  2016    ME      1058.0       830.0     754.0        0.908434   
20  2016    MD      4623.0      3114.0    2737.0        0.878934   
21  2016    MA      5374.0      3660.0    3315.0        0.905738   
22  2016    MI      7624.0      5434.0    4713.0        0.867317   
23  2016    MN      4190.0      3055.0    2738.0        0.896236   
24  2016    MS      2203.0      1725.0    1470.0        0.852174   
25  2016    MO      4626.0      3333.0    2906.0        0.871887   
26  2016    MT       798.0       581.0     521.0        0.896730   
27  2016    NE      1407.0      1008.0     893.0        0.885913   
28  2016    NV      2234.0      1371.0    1195.0        0.871627   
29  2016    NH      1044.0       763.0     698.0        0.914810   
30  2016    NJ      6862.0      4165.0    3665.0        0.879952   
31  2016    NM      1547.0       916.0     765.0        0.835153   
32  2016    NY     15506.0      9142.0    7869.0        0.860753   
33  2016    NC      7631.0      5194.0    4700.0        0.904890   
34  2016    ND       583.0       424.0     362.0        0.853774   
35  2016    OH      8811.0      6128.0    5408.0        0.882507   
36  2016    OK      2923.0      1861.0    1555.0        0.835572   
37  2016    OR      3185.0      2147.0    1942.0        0.904518   
38  2016    PA      9980.0      6909.0    6008.0        0.869590   
39  2016    RI       836.0       538.0     464.0        0.862454   
40  2016    SC      3733.0      2575.0    2233.0        0.867184   
41  2016    SD       631.0       437.0     362.0        0.828375   
42  2016    TN      5057.0      3251.0    2630.0        0.808982   
43  2016    TX     20172.0     11724.0    9626.0        0.821051   
44  2016    UT      2096.0      1398.0    1234.0        0.882690   
45  2016    VT       500.0       351.0     305.0        0.868946   
46  2016    VA      6343.0      4399.0    3973.0        0.903160   
47  2016    WA      5592.0      3906.0    3382.0        0.865847   
48  2016    WV      1434.0       913.0     723.0        0.791895   
49  2016    WI      4465.0      3323.0    3068.0        0.923262   
50  2016    WY       436.0       304.0     277.0        0.911184   
51  2016    US    245502.0    157596.0  137537.0        0.872719   

    Norm_Voter_Pop  
0         0.563627  
1         0.594595  
2         0.532910  
3         0.560018  
4         0.482237  
5         0.638142  
6         0.574846  
7         0.572016  
8         0.687161  
9         0.529441  
10        0.556779  
11        0.432331  
12        0.583333  
13        0.588193  
14        0.560345  
15        0.607352  
16        0.580299  
17        0.552569  


# Add to sql db

In [21]:
# import psycopg2
# from sqlalchemy import create_engine

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    database=DATABASE,
    user=USER,
    password=PASSWORD,
    host=HOST,
    port=PORT
)

# Create a sqlalchemy engine
engine = create_engine(f"postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

# Upload the data frames to the database using the voters table
df_votes.to_sql("voters", engine, if_exists="append", index=False)
print("Votes table loaded successfully")

# Commit the changes to the database
conn.commit()

# Close the cursor and database connection
cursor.close()
conn.close()


Votes table loaded successfully


In [22]:
# import psycopg2
# import pandas as pd

# Replace the values below with your database credentials
DATABASE = "electiondb"
USER = "postgres"
PASSWORD = "YourPassword"
HOST = "localhost"
PORT = "5432"

# Connect to the PostgreSQL database
conn = psycopg2.connect(database=DATABASE, user=USER, password=PASSWORD, host=HOST, port=PORT)

# Query the table and store the results in a Pandas dataframe
df = pd.read_sql_query("SELECT * FROM voters", conn)

# Print the dataframe to verify that it contains data
print(df)

# Close the database connection
conn.close()


     Year State  Population  Registered   Voted  Norm_Voter_Reg   
0    2020    AL      3769.0        2527    2247        0.889197  \
1    2020    AK       528.0         383     330        0.861619   
2    2020    AZ      5638.0        3878    3649        0.940949   
3    2020    AR      2283.0        1361    1186        0.871418   
4    2020    CA     30342.0       18001   16893        0.938448   
..    ...   ...         ...         ...     ...             ...   
151  2016    WA      5592.0        3906    3382        0.865847   
152  2016    WV      1434.0         913     723        0.791895   
153  2016    WI      4465.0        3323    3068        0.923262   
154  2016    WY       436.0         304     277        0.911184   
155  2016    US    245502.0      157596  137537        0.872719   

     Norm_Voter_Pop  
0          0.596179  
1          0.625000  
2          0.647215  
3          0.519492  
4          0.556753  
..              ...  
151        0.604793  
152        0.504184